# What drives the price of a car?

![](images/kurt.jpeg)

**OVERVIEW**

In this application, you will explore a dataset from Kaggle. The original dataset contained information on 3 million used cars. The provided dataset contains information on 426K cars to ensure speed of processing.  Your goal is to understand what factors make a car more or less expensive.  As a result of your analysis, you should provide clear recommendations to your client -- a used car dealership -- as to what consumers value in a used car.

### CRISP-DM Framework

<center>
    <img src = images/crisp.png width = 50%/>
</center>


To frame the task, throughout our practical applications, we will refer back to a standard process in industry for data projects called CRISP-DM.  This process provides a framework for working through a data problem.  Your first step in this application will be to read through a brief overview of CRISP-DM [here](https://mo-pcco.s3.us-east-1.amazonaws.com/BH-PCMLAI/module_11/readings_starter.zip).  After reading the overview, answer the questions below.

### Business Understanding

From a business perspective, we are tasked with identifying key drivers for used car prices.  In the CRISP-DM overview, we are asked to convert this business framing to a data problem definition.  Using a few sentences, reframe the task as a data task with the appropriate technical vocabulary. 

### Data Understanding

After considering the business understanding, we want to get familiar with our data.  Write down some steps that you would take to get to know the dataset and identify any quality issues within.  Take time to get to know the dataset and explore what information it contains and how this could be used to inform your business understanding.

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import mutual_info_classif
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, OneHotEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer, make_column_selector

from sklearn import set_config

### utility functions
- read_data(): read vehicles data
- function find_discrete_columns: find discrete columns which need to be encoded by OneHotEncoder or OrdinalEncoder
- function find_numerical_columns: find numerical columns which may have different default values for nans
- nan_analysis: return map of column name and the percentage of nan values if present in that column

In [26]:
def read_data():
    df=pd.read_csv('data/vehicles.csv')
    return df

def find_discrete_columns(df):
    # Step 1: Identify discrete/categorical columns
    discrete_columns = ['model','region']
    
    for column in df.columns:
        # Check if column is of object type (typically strings) or has few unique values
        #print(f"Found discrete column: {column} with {len(df[column].unique())} unique values")
        if len(df[column].unique()) < 60:  # Adjust threshold as needed
            discrete_columns.append(column)
            #print(f"Found discrete column: {column} with {len(df[column].unique())} unique values")
    return discrete_columns

def find_numerical_columns(df):
    # Step 1: Identify discrete/categorical columns
    numerical_columns = []
    
    for column in df.columns:
        # Check if column is of object type (typically strings) or has few unique values
        if df[column].dtype in ['int64', 'float64']:  # Adjust threshold as needed
            numerical_columns.append(column)
            #print(f"Found numerical column: {column}")
    return numerical_columns

def nan_analysis(df):
    # Check for missing values in the DataFrame
    
    has_nan = df.isnull().values.any()
    if has_nan:
        total_rows = len(df)
        # Calculate percentage of NaN values per column
        nan_percentage = (df.isnull().sum() / total_rows) * 100

        # Filter only columns that have NaN values
        nan_columns = nan_percentage[nan_percentage > 0]
        #print('Percentage of NaN values per column if any NaN exists:')
        nan_columns.map(lambda x: str(x)+'%')
        return nan_columns
    else:
        print('No NaN values in the DataFrame.')
        return {}

### load data and preview the data

In [37]:
df = read_data()
df.info()
df.tail(10)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
426870,7301592119,wyoming,22990,2020.0,hyundai,sonata se sedan 4d,good,NaN,gas,3066.0,clean,other,5NPEG4JAXLH051710,fwd,NaN,sedan,blue,wy
426871,7301591639,wyoming,17990,2018.0,kia,sportage lx sport utility 4d,good,NaN,gas,34239.0,clean,other,KNDPMCAC7J7417329,NaN,NaN,SUV,NaN,wy
426872,7301591201,wyoming,32590,2020.0,mercedes-benz,c-class c 300,good,NaN,gas,19059.0,clean,other,55SWF8DB6LU325050,rwd,NaN,sedan,white,wy
426873,7301591202,wyoming,30990,2018.0,mercedes-benz,glc 300 sport,good,NaN,gas,15080.0,clean,automatic,WDC0G4JB6JV019749,rwd,NaN,other,white,wy
426874,7301591199,wyoming,33590,2018.0,lexus,gs 350 sedan 4d,good,6 cylinders,gas,30814.0,clean,automatic,JTHBZ1BLXJA012999,rwd,NaN,sedan,white,wy
426875,7301591192,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,wy
426876,7301591187,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,wy
426877,7301591147,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,wy
426878,7301591140,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,wy
426879,7301591129,wyoming,30590,2019.0,bmw,4 series 430i gran coupe,good,NaN,gas,22716.0,clean,other,WBA4J1C58KBM14708,rwd,NaN,coupe,NaN,wy


### Data observation:
    - Data columns not related to price from business point of view and should be dropped before further analysis:
        id
        VIN
    - Some data columns have nan values which need to be pre-processed before further analysis
    - cylinders column contains a value "other" which seems to be a good choice for nans when fillna() is called 


In [28]:
discrete_columns = find_discrete_columns(df)
print('discrete_columns\\n',discrete_columns)

numerical_columns = find_numerical_columns(df)
print('numerical_columns:\\n', numerical_columns)

discrete_columns\n ['model', 'region', 'manufacturer', 'condition', 'cylinders', 'fuel', 'title_status', 'transmission', 'drive', 'size', 'type', 'paint_color', 'state']
numerical_columns:\n ['id', 'price', 'year', 'odometer']


### nan data analysis

In [29]:


print(nan_analysis(df))

year             0.282281
manufacturer     4.133714
model            1.236179
condition       40.785232
cylinders       41.622470
fuel             0.705819
odometer         1.030735
title_status     1.930753
transmission     0.598763
VIN             37.725356
drive           30.586347
size            71.767476
type            21.752717
paint_color     30.501078
dtype: float64


### discret value analysis
    - "other" seems to good value for nans for all the discret columns including the following with "other" already:
        - transmission: "other"
        - fuel: "other'
        - model: "other"
    - "other" will be used for discret columns when fillna() is called
    - year & odometer are the only numerical columns have nan values. Different default values will be tried to see which one makes sense:
        - year : mdian value for that whole data set? 
        - odometer : median value for that year?   
        


In [30]:
for col in np.intersect1d( discrete_columns, nan_analysis(df).keys()):
    print(f"{col} ",df[col].unique(), "\n")


condition  [nan 'good' 'excellent' 'fair' 'like new' 'new' 'salvage'] 

cylinders  [nan '8 cylinders' '6 cylinders' '4 cylinders' '5 cylinders' 'other'
 '3 cylinders' '10 cylinders' '12 cylinders'] 

drive  [nan 'rwd' '4wd' 'fwd'] 

fuel  [nan 'gas' 'other' 'diesel' 'hybrid' 'electric'] 

manufacturer  [nan 'gmc' 'chevrolet' 'toyota' 'ford' 'jeep' 'nissan' 'ram' 'mazda'
 'cadillac' 'honda' 'dodge' 'lexus' 'jaguar' 'buick' 'chrysler' 'volvo'
 'audi' 'infiniti' 'lincoln' 'alfa-romeo' 'subaru' 'acura' 'hyundai'
 'mercedes-benz' 'bmw' 'mitsubishi' 'volkswagen' 'porsche' 'kia' 'rover'
 'ferrari' 'mini' 'pontiac' 'fiat' 'tesla' 'saturn' 'mercury'
 'harley-davidson' 'datsun' 'aston-martin' 'land rover' 'morgan'] 

model  [nan 'sierra 1500 crew cab slt' 'silverado 1500' ... 'gand wagoneer'
 '96 Suburban' 'Paige Glenbrook Touring'] 

paint_color  [nan 'white' 'blue' 'red' 'black' 'silver' 'grey' 'brown' 'yellow'
 'orange' 'green' 'custom' 'purple'] 

size  [nan 'full-size' 'mid-size' 'compact' 

### Data Preparation

After our initial exploration and fine-tuning of the business understanding, it is time to construct our final dataset prior to modeling.  Here, we want to make sure to handle any integrity issues and cleaning, the engineering of new features, any transformations that we believe should happen (scaling, logarithms, normalization, etc.), and general preparation for modeling with `sklearn`. 

### Drop columns not related to price from business point of view, such as id, VIN

In [31]:
df.drop(columns=['id','VIN'], inplace=True)

### fill nan discret columns as "other"

In [32]:
for col in discrete_columns:
    df = df.fillna({
        col: 'other',
    })

df = df.fillna(
    {
        "year": df["year"].median(),
        "odometer": df.groupby('year')['odometer'].transform(lambda x: x.fillna(x.mean()))
    }

)
df['odometer']= df.groupby('year')['odometer'].transform(lambda x: x.fillna(x.mean()))
df = df.fillna(
    {
        "odometer": df["odometer"].median(),
    }

)

nan_analysis(df)

No NaN values in the DataFrame.


{}

### Modeling

With your (almost?) final dataset in hand, it is now time to build some models.  Here, you should build a number of different regression models with the price as the target.  In building your models, you should explore different parameters and be sure to cross-validate your findings.

- Use of multiple regression models
    - Linear regression
    - multiple regression
    - Ridge regression
    - Lasso regression

- Cross-validation of models
    - k-fold cross-validation
- Grid search hyperparameters
   
- Appropriate interpretation of coefficients in models
- Appropriate interpretation of evaluation metric
- Clear identification of evaluation metric
- Clear rationale for use of given evaluation metric

In [33]:
print(df.columns)


Index(['region', 'price', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')


In [39]:
X = df.drop(['price'], axis = 1)
print(X.columns)
y = df['price']
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=22)
scaler = MinMaxScaler()
cols_to_normalize = ['odometer', 'year']
X_train[cols_to_normalize] = scaler.fit_transform(X_train[cols_to_normalize])
X_test[cols_to_normalize] = scaler.transform(X_test[cols_to_normalize])

Index(['id', 'region', 'year', 'manufacturer', 'model', 'condition',
       'cylinders', 'fuel', 'odometer', 'title_status', 'transmission', 'VIN',
       'drive', 'size', 'type', 'paint_color', 'state'],
      dtype='object')


In [ ]:



onehot_cols = discrete_columns.copy()
onehot_cols.remove('condition')


ordinal_ohe_transformer = make_column_transformer(    
    (OneHotEncoder(drop = 'if_binary', handle_unknown='ignore'), onehot_cols), 
    (OrdinalEncoder(categories = [['other','salvage','fair', 'good', 'excellent', 'like new', 'new']], handle_unknown='use_encoded_value', unknown_value=-1),['condition']), 
    remainder='passthrough')


ordinal_ohe_transformer.fit_transform(X_train[discrete_columns])

s=""

pipe_1 = Pipeline([
    ('transformer', ordinal_ohe_transformer),  # Standardize the data
    ('linreg', LinearRegression())  # Train Linear Regression model
])
print(pipe_1.named_steps)
pipe_1.fit(X_train[X.columns], y_train)
pred_train = pipe_1.predict(X_train)
pipe_1_train_mse = mean_squared_error (y_train, pred_train )




pred_test = pipe_1.predict(X_test)
pipe_1_test_mse = mean_squared_error (y_test, pred_test )
# Answer check
print(pipe_1.named_steps)
print(f'Train MSE: {pipe_1_train_mse: .2f}')
print(f'Test MSE: {pipe_1_test_mse: .2f}')

ValueError: A given column is not a column of the dataframe

### Evaluation

With some modeling accomplished, we aim to reflect on what we identify as a high-quality model and what we are able to learn from this.  We should review our business objective and explore how well we can provide meaningful insight into drivers of used car prices.  Your goal now is to distill your findings and determine whether the earlier phases need revisitation and adjustment or if you have information of value to bring back to your client.

### Deployment

Now that we've settled on our models and findings, it is time to deliver the information to the client.  You should organize your work as a basic report that details your primary findings.  Keep in mind that your audience is a group of used car dealers interested in fine-tuning their inventory.